In [1]:
import FlowCal

In [2]:
# Wrong URL
#!wget https://github.com/taborlab/FlowCal/blob/master/examples/FCFiles/sample001.fcs
# Correct URL
#!wget https://github.com/taborlab/FlowCal/raw/master/examples/FCFiles/sample001.fcs

We use `urllib` to download a few example `.fcs` files from the github repo of `FlowCal`:

In [3]:
import urllib

mother_url = (
    "https://github.com/taborlab/"
    "FlowCal/raw/master/examples/FCFiles/"
)
filenames = [f'sample{i:003d}.fcs' for i in (1,6,7,8)]
for name in filenames:
    with urllib.request.urlopen(mother_url+name) as response:
        with open(name, "wb") as f:
            f.write(response.read())
        print(f'Finish downloading {name}')

Finish downloading sample001.fcs
Finish downloading sample006.fcs
Finish downloading sample007.fcs
Finish downloading sample008.fcs


In [4]:
!ls *.fcs

sample001.fcs  sample006.fcs  sample007.fcs  sample008.fcs


In [5]:
s = FlowCal.io.FCSData("sample001.fcs")
s

FCSData([[   0,  336,  527, ...,  626,   88,   75],
         [   0,  353,  550, ...,  967,  249, 1023],
         [   0,  367,  528, ...,  969,  251, 1023],
         ...,
         [ 791,  339,  518, ...,  228,    0,    0],
         [ 791,  334,  476, ...,  629,  151,   84],
         [ 791,  324,  576, ...,  617,  111,   72]], dtype=uint32)

In [6]:
import numpy as np

issubclass(FlowCal.io.FCSData, np.ndarray)

True

In [7]:
print([string for string in dir(s) if not string.startswith("_")])

['T', 'acquisition_end_time', 'acquisition_start_time', 'acquisition_time', 'all', 'amplification_type', 'amplifier_gain', 'analysis', 'any', 'argmax', 'argmin', 'argpartition', 'argsort', 'astype', 'base', 'byteswap', 'channel_labels', 'channels', 'choose', 'clip', 'compress', 'conj', 'conjugate', 'copy', 'ctypes', 'cumprod', 'cumsum', 'data', 'data_type', 'detector_voltage', 'diagonal', 'dot', 'dtype', 'dump', 'dumps', 'fill', 'flags', 'flat', 'flatten', 'getfield', 'hist_bins', 'imag', 'infile', 'item', 'itemset', 'itemsize', 'max', 'mean', 'min', 'nbytes', 'ndim', 'newbyteorder', 'nonzero', 'partition', 'prod', 'ptp', 'put', 'range', 'ravel', 'real', 'repeat', 'reshape', 'resize', 'resolution', 'round', 'searchsorted', 'setfield', 'setflags', 'shape', 'size', 'sort', 'squeeze', 'std', 'strides', 'sum', 'swapaxes', 'take', 'text', 'time_step', 'tobytes', 'tofile', 'tolist', 'tostring', 'trace', 'transpose', 'var', 'view']


In [8]:
s.shape

(33024, 8)

- The first dimension is called **event**
- The 2nd **channel**

One could slice through a `FCSData` like one does to `ndarray`. For example, let's take just a few channels.

In [9]:
s_sub_ch = s[:, [3,4,5]]
s_sub_ch.shape

(33024, 3)

`FCSData` implements a few other convenience methods for us to more easily manipulate the channels:

In [10]:
print(f'{s.channels}')
print(f'{s_sub_ch.channels}')

('TIME', 'FSC', 'SSC', 'FL1', 'FL2', 'FL3', 'FSCW', 'FSCA')
('FL1', 'FL2', 'FL3')


So the channels `[3,4,5]` we just took simply correspond to the fluorescence channels `FL1`, `FL2` and `FL3`.

In [11]:
s_sub_ch2 = s[:, ["FL1", "FL2", "FL3"]]
s_sub_ch2.shape

(33024, 3)

In [12]:
np.array_equal(s_sub_ch, s_sub_ch2)

True

In [13]:
s_sub_ch2.channels

('FL1', 'FL2', 'FL3')

`FCSData` contains more acquisition information, such as the acquisition time, amplifier type, and the detector voltage of each channel. For more information, consult the documentation of [`FlowCal.io.FCSData`](https://taborlab.github.io/FlowCal/reference/FlowCal.io.html#FlowCal.io.FCSData).

## The Acrynoms of the Channels

From the tutorials we know at least the following

- `"FL{i}"` for `i = 1, 2, 3`: Fluorescence Channels
- `"FSC"`: Forward Scatter Channel
- `"SSC"`: Side Scatter Channel

In [14]:
s.acquisition_time

79.10000000000001

In [15]:
s.acquisition_start_time, s.acquisition_end_time

(datetime.datetime(2018, 3, 2, 2, 24, 47),
 datetime.datetime(2018, 3, 2, 2, 26, 8))

In [16]:
s.acquisition_end_time - s.acquisition_start_time

datetime.timedelta(seconds=81)

In [17]:
s[:10, ["TIME"]]

FCSData([[0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0]], dtype=uint32)

In [18]:
s[-10:, ["TIME"]]

FCSData([[791],
         [791],
         [791],
         [791],
         [791],
         [791],
         [791],
         [791],
         [791],
         [791]], dtype=uint32)

So the `"TIME"` channel refers, with no surprise, to time. More precisely, the **acquisition time**. It seems to have the unit of deci-second in this particular case of `sample0001.fcs`.

We can probably obtain each event's (absolute) acquisition time by adding together the `"TIME"` column with `s.acquisition_start_time`.